In [2]:
import torch
from torch.utils.data import DataLoader
import importlib
import numpy as np
import torch.nn as nn
import time
from tqdm.notebook import tqdm
import copy
import os

Run the following code block every time you modify utils

In [3]:
from utils import preprocess, dataset, network, train
importlib.reload(preprocess)
importlib.reload(dataset)
importlib.reload(network)
importlib.reload(train)

<module 'utils.train' from '/project/ruishanl_1185/EHR-Distillation/utils/train.py'>

1. Compute statistics from training set.

In [4]:
continuous_avgs_train, continuous_stds_train, categorical_modes_train =  preprocess.compute_feature_statistics(
    ts_dir="./data/mimic3/benchmark/in-hospital-mortality/train/",
    feature_dict=preprocess.mimic3_benchmark_variable_dict
    )

Continuous features' averages: 
diastolic_blood_pressure     61.047942
fraction_inspired_oxygen      0.540138
glucose                     147.949874
heart_rate                   86.976837
height                      168.557096
mean_blood_pressure          78.569455
oxygen_saturation            97.755870
respiratory_rate             21.426361
systolic_blood_pressure     119.722756
temperature                  37.009114
weight                       83.055615
ph                            7.279231
dtype: float64
Continuous features' standard deviations: 
diastolic_blood_pressure     223.959153
fraction_inspired_oxygen       0.196874
glucose                     2010.638970
heart_rate                    19.941287
height                        14.856180
mean_blood_pressure          147.585392
oxygen_saturation            951.247931
respiratory_rate            2282.526849
systolic_blood_pressure       29.291774
temperature                    9.685635
weight                        24.704757
ph

2. Clean all data by resampling.
Running this block once is enough.
Cleaned data will be saved at ./data/mimic3/ihm_preliminary/

In [7]:
preprocess.preprocess_ihm_timeseries_files(
    ts_dir="./data/mimic3/benchmark/in-hospital-mortality/train/",
    output_dir="./data/mimic3/ihm_preliminary/train/",
    feature_dict=preprocess.mimic3_benchmark_variable_dict,
    normal_value_dict=continuous_avgs_train|categorical_modes_train
    )
preprocess.preprocess_ihm_timeseries_files(
    ts_dir="./data/mimic3/benchmark/in-hospital-mortality/test/",
    output_dir="./data/mimic3/ihm_preliminary/test/",
    feature_dict=preprocess.mimic3_benchmark_variable_dict,
    normal_value_dict=continuous_avgs_train|categorical_modes_train
    )

100%|██████████| 3236/3236 [01:28<00:00, 36.45it/s]


3. Prepare real training set and test set.

In [5]:
# Load datasets

categorical_numcls = {  # how many classes are there for categorical classes
    "capillary_refill_rate": 2,
    "glascow_coma_scale_eye_opening": 4,
    "glascow_coma_scale_motor_response": 6,
    "glascow_coma_scale_total": 13,
    "glascow_coma_scale_verbal_response": 5,
}

train_set = dataset.IHMPreliminaryDatasetReal(
    dir="./data/mimic3/ihm_preliminary/train/",
    dstype="train",
    avg_dict=continuous_avgs_train,
    std_dict=continuous_stds_train,
    numcls_dict=categorical_numcls
    )
print(f"First item in the dataset: \n{train_set[0]}")
print(f"Feature tensor shape: {train_set[0][0].shape}")

test_set = dataset.IHMPreliminaryDatasetReal(
    dir="./data/mimic3/ihm_preliminary/test/",
    dstype="test",
    avg_dict=continuous_avgs_train,
    std_dict=continuous_stds_train,
    numcls_dict=categorical_numcls
    )
print(f"First item in the dataset: \n{test_set[0]}")
print(f"Feature tensor shape: {test_set[0][0].shape}")

input_shape = train_set[0][0].shape

First item in the dataset: 
(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]]), tensor(0))
Feature tensor shape: torch.Size([48, 59])
First item in the dataset: 
(tensor([[1.0000, 0.0000, 0.0000,  ..., 1.0000, 0.0391, 1.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0391, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0391, 0.0000],
        ...,
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0521, 1.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0521, 1.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0521, 0.0000]]), tensor(0))
Feature tensor shape: torch.Size([48, 59])


4. See label distribution in training set.

In [6]:
label_0_cnt = 0
label_1_cnt = 1
for _, label in train_set:
    if label > 0.5:
        label_1_cnt += 1
    else:
        label_0_cnt += 1
print(f"Label 0 ratio: {label_0_cnt / (label_0_cnt + label_1_cnt)}")
print(f"Label 1 ratio: {label_1_cnt / (label_0_cnt + label_1_cnt)}")

KeyboardInterrupt: 

5. Define device

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


6. Evaluate model on original downstream task (in-hospital mortality prediction /binary classification)

In [8]:
# define hyperparams

ihm_epoch = 100
ihm_batch_size = 256
ihm_lr = 0.001

In [14]:
# train
num_workers = 60 # slightly less than CPU(s) line running lscpu
pkl_save_dir = "./saved_data/ihm_model/"
if not os.path.exists(pkl_save_dir):
    os.makedirs(pkl_save_dir)

train_loader = DataLoader(train_set, ihm_batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_set, ihm_batch_size, num_workers=num_workers)

model = network.IHMPreliminary1DCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=ihm_lr)
criterion = nn.CrossEntropyLoss()


pbar = tqdm(range(ihm_epoch), desc="Training on original task")
min_loss = float("inf")
for e in pbar:
    train_loss, train_acc = train.epoch("train", train_loader, model, criterion, optimizer, device)
    if train_loss < min_loss:
        filename = f'ihm_1dcnn_e{e}_loss{train_loss:.4f}.pt'
        file_path = os.path.join(pkl_save_dir, filename)

        # Remove the previous checkpoint if it exists
        # existing_pts = [f for f in os.listdir(pkl_save_dir) if f.startswith('ihm_') and f.endswith('.pt')]
        # for f in existing_pts:
        #     os.remove(os.path.join(pkl_save_dir, f))
        torch.save({
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, file_path)
    test_loss, test_acc = train.epoch("test", test_loader, model, criterion, device=device)
    pbar.set_description(f"Training on original task, epoch {e}\ntrain loss = {train_loss}, train acc = {train_acc}\ntest loss = {test_loss}, test acc ={test_acc}")

Training on original task:   0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

7. Start to distill EHR dataset.

In [66]:
# define hyper params

train_it = 100
lr_data = 0.01
lr_net = 0.01
episodes_per_cls = 10
num_outer_loop = 10
num_inner_loop = 50
real_batch_size = 256
syn_batch_size = 256


In [47]:
# initialize synthetic data

episodes_syn = torch.randn(size=(2*episodes_per_cls, input_shape[0], input_shape[1]), dtype=torch.float, requires_grad=True) # device is ignored by far
labels_syn = torch.tensor(np.array([np.ones(episodes_per_cls)*i for i in (0, 1)]), dtype=torch.long, requires_grad=False).view(-1) # 1-D, length = episodes_per_cls * 2

In [48]:
# define training optimizers and criterion
optimizer_ts = torch.optim.SGD([episodes_syn,], lr=lr_data, momentum=0.5) # optimizer for synthetic data
optimizer_ts.zero_grad()
criterion = nn.CrossEntropyLoss()
print("Ready for training")

Ready for training


In [67]:
# Train
pkl_save_dir = "./saved_data"
if not os.path.exists(pkl_save_dir):
    os.makedirs(pkl_save_dir)
pbar = tqdm(range(train_it), desc="Training iteration")
for it in pbar:

    # Evaluate synthetic data
    # TODO

    # Train synthetic data
    torch.random.manual_seed(int(time.time() * 1000) % 100000) # random init network
    net = network.IHMPreliminary1DCNN(input_shape=input_shape)
    net.train()
    net_params = list(net.parameters())

    optimizer_net = torch.optim.SGD(net.parameters(), lr=lr_net)
    optimizer_net.zero_grad()
    loss_avg = 0

    for ol in range(num_outer_loop):
        # update synthetic data
        loss = torch.tensor(0.0)
        for cls in (0, 1):
            ts_real, lab_real = train_set.random_sample_from_class(n_samples=real_batch_size, cls=cls)
            ts_syn = episodes_syn[cls*episodes_per_cls: (cls+1)*episodes_per_cls]
            lab_syn = labels_syn[cls*episodes_per_cls: (cls+1)*episodes_per_cls]

            out_real = net(ts_real)
            loss_real = criterion(out_real, lab_real)
            grad_real = torch.autograd.grad(loss_real, net_params)
            grad_real = [_.detach().clone() for _ in grad_real]

            out_syn = net(ts_syn)
            loss_syn = criterion(out_syn, lab_syn)
            grad_syn = torch.autograd.grad(loss_syn, net_params, create_graph=True) # create_graph: will be used to compute higher-order derivatives

            # compute gradient matching loss, here using MSE, instead of the one proposed in DCwMG because it's too complicated
            dis = torch.tensor(0.0)
            grad_real_vec = []
            grad_syn_vec = []
            for ig in range(len(grad_real)):
                grad_real_vec.append(grad_real[ig].reshape((-1)))
                grad_syn_vec.append(grad_syn[ig].reshape((-1)))
            grad_real_vec = torch.cat(grad_real_vec, dim=0)
            grad_syn_vec = torch.cat(grad_syn_vec, dim=0)
            dis = torch.sum((grad_syn_vec - grad_real_vec)**2)

            loss += dis
        
        optimizer_ts.zero_grad()
        loss.backward()
        optimizer_ts.step()
        loss_avg += loss.item()

        if ol == num_outer_loop - 1:
            break

        # update network
        episodes_syn_train, labels_syn_train = copy.deepcopy(episodes_syn.detach()), copy.deepcopy(labels_syn.detach())  # avoid any unaware modification
        syn_dataset = dataset.TensorDataset(episodes_syn_train, labels_syn_train)
        train_loader = DataLoader(syn_dataset, syn_batch_size, shuffle=True)
        for il in range(num_inner_loop):
            train.epoch("train", train_loader, net, optimizer_net, criterion)

    loss_avg /= (2 * num_outer_loop)
    pbar.set_description(f"Training iteration, average loss = {loss_avg}")

torch.save({"data": (copy.deepcopy(episodes_syn.detach()), copy.deepcopy(labels_syn.detach()))},
           os.path.join(pkl_save_dir, "distilled_dataset.pt")
           )



Training iteration:   0%|          | 0/100 [00:00<?, ?it/s]